In [ ]:
!pip install selenium

In [ ]:
!pip install gspread oauth2client pandas

In [ ]:
# -*- coding: utf-8 -*-
import time
import csv
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import pandas as pd

# Google Colab で Google ドライブをマウントする場合（今回は不要の場合はコメントアウト可）
# from google.colab import drive
# drive.mount('/content/drive')

def extract_card_data(card_html):
    """
    1件分の募集カード HTML から各情報を抽出する関数。
    取得項目:
      - company: 企業名
      - deadline_date: 締切日（例: 「本日」「明日」または "2/16"）
      - deadline_time: 締切時間（例: "17:00" または "23:59"）
      - content: 募集タイトル
      - schedule: 日程
      - location: 開催場所
      - url: 募集詳細ページの URL
    """
    soup = BeautifulSoup(card_html, "html.parser")

    # ① 企業名の抽出
    company_div = soup.find("div", class_="mantine-Text-root text-md text-black w-full cursor-pointer hover:opacity-50 pl-12 pr-8 line-clamp-2 mantine-1d564l0")
    company = company_div.get_text(strip=True) if company_div else ""

    # ② 締切情報の抽出（2種類のパターンに対応）
    deadline_date = ""
    deadline_time = ""
    # まず、bg-deadline-notifcation のコンテナを探す
    deadline_container = soup.find("div", class_="bg-deadline-notifcation")
    if deadline_container:
        inner_div = deadline_container.find("div", class_="mantine-1avyp1d")
        if inner_div:
            spans = inner_div.find_all("span")
            if len(spans) >= 2:
                deadline_date = spans[0].get_text(strip=True)
                deadline_time = spans[1].get_text(strip=True)
    else:
        # 見つからなかった場合、text-deadline-primary のコンテナを探す
        deadline_container = soup.find("div", class_="text-deadline-primary")
        if deadline_container:
            # ②の例では、内部に <div class="flex justify-center items-baseline mantine-1avyp1d"> がある
            inner_div = deadline_container.find("div", class_="flex")
            if inner_div:
                spans = inner_div.find_all("span")
                if len(spans) >= 3:
                    # 1番目の span を締切日、3番目の span を締切時間とする（2番目は曜日など）
                    deadline_date = spans[0].get_text(strip=True)
                    deadline_time = spans[2].get_text(strip=True)
                elif len(spans) >= 2:
                    deadline_date = spans[0].get_text(strip=True)
                    deadline_time = spans[1].get_text(strip=True)

    # ③ 募集タイトル（内容）の抽出
    content_div = soup.find("div", class_="font-bold pc:text-md text-sm line-clamp-2")
    content = content_div.get_text(strip=True) if content_div else ""

    # ④ 募集詳細ページの URL の抽出
    # カード内の最初の <a> タグの href 属性を利用
    a_tag = soup.find("a", href=True)
    url_value = ""
    if a_tag:
        url_value = a_tag["href"]
        if url_value.startswith("/"):
            url_value = "https://gaishishukatsu.com" + url_value

    # ⑤ 日程と開催場所の抽出
    schedule = ""
    location = ""
    parent_divs = soup.find_all("div", class_="inline pc:text-base text-sm")
    for parent in parent_divs:
        label_div = parent.find("div", class_="border-[1px] border-black-2 rounded-full px-8 inline-block line-clamp-1 mr-8")
        if label_div:
            label_text = label_div.get_text(strip=True)
            value_div = parent.find("div", class_="inline-block whitespace-normal line-clamp-1")
            if value_div:
                if "日程" in label_text:
                    schedule = value_div.get_text(strip=True)
                elif "場所" in label_text:
                    location = value_div.get_text(strip=True)

    return {
        "company": company,
        "deadline_date": deadline_date,
        "deadline_time": deadline_time,
        "content": content,
        "schedule": schedule,
        "location": location,
        "url": url_value
    }

def scrape_all_pages():
    chrome_options = Options()
    chrome_options.add_argument("--headless")
    chrome_options.add_argument("--disable-gpu")
    chrome_options.add_argument("--no-sandbox")

    driver = webdriver.Chrome(options=chrome_options)

    results = []
    page = 1
    while True:
        url = f"https://gaishishukatsu.com/recruiting_info?order=deadline&gy=2027&allgy=false&page={page}"
        print(f"Processing page {page}: {url}")
        driver.get(url)

        try:
            WebDriverWait(driver, 15).until(
                EC.presence_of_element_located(
                    (By.CSS_SELECTOR, "div.bg-white.rounded-\\[16px\\].shadow-card.mb-16.mantine-1avyp1d")
                )
            )
        except Exception as e:
            print(f"Warning: ページ {page} で募集カード待機中エラー:", e)

        time.sleep(2)  # レンダリング待ち
        html = driver.page_source
        soup = BeautifulSoup(html, "html.parser")

        # 「一致する募集情報は見つからなかった」メッセージがある場合、ループ終了
        no_info_div = soup.find("div", class_="bg-white mb-[10px] px-20 py-[10px] text-base text-black-1 mantine-1avyp1d")
        if no_info_div and "一致する募集情報は見つからなかった" in no_info_div.get_text():
            print(f"ページ {page} で募集情報が見つからなかったためループ終了")
            break

        # 募集カードの抽出
        cards = soup.find_all("div", class_="bg-white rounded-[16px] shadow-card mb-16 mantine-1avyp1d")
        print(f"ページ {page} の募集カード数:", len(cards))

        # 募集カードが 0 件の場合は処理終了
        if len(cards) == 0:
            print(f"ページ {page} で募集カードが見つからなかったため処理を終了します。")
            break

        for card in cards:
            card_html = str(card)
            data = extract_card_data(card_html)
            results.append(data)
            print("書き込み:", data)

        page += 1
        time.sleep(1)

    driver.quit()
    return results

def save_results_to_csv(results, csv_filename="recruitment_info_all_pages_gaishishukatsu.csv"):
    """
    指定のファイル名（保存先パスを指定しない場合は現在の作業ディレクトリ）に CSV を保存する
    """
    df = pd.DataFrame(results)
    df.to_csv(csv_filename, index=False, encoding="utf-8-sig")
    print("CSV saved to:", csv_filename)

# メイン処理
results = scrape_all_pages()
save_results_to_csv(results)

In [ ]:
import gspread
import pandas as pd
from oauth2client.service_account import ServiceAccountCredentials

def update_spreadsheet_from_csv():
    """
    CSVファイル (recruitment_info_all_pages_gaishishukatsu.csv) の内容を
    Google スプレッドシート「選考締め切り情報」の「外資就活」シートに反映する。
    """

    # 1. 使用するスコープを定義
    scope = [
        "https://spreadsheets.google.com/feeds",
        "https://www.googleapis.com/auth/spreadsheets",
        "https://www.googleapis.com/auth/drive.file",
        "https://www.googleapis.com/auth/drive"
    ]

    # 2. credentials.json (サービスアカウントの認証情報) を使って認証
    creds = ServiceAccountCredentials.from_json_keyfile_name("credentials.json", scope)
    client = gspread.authorize(creds)

    # 3. 対象スプレッドシートとシートを取得
    spreadsheet = client.open("選考締め切り情報")  # スプレッドシート名
    worksheet = spreadsheet.worksheet("外資就活")  # シート名

    # 4. CSVファイルを pandas で読み込み
    csv_filename = "recruitment_info_all_pages_gaishishukatsu.csv"
    df = pd.read_csv(csv_filename, encoding="utf-8-sig")

    # 5. DataFrame をリスト形式に変換（先頭にヘッダー行を含む）
    data = [df.columns.tolist()] + df.values.tolist()

    # 6. シートの内容をクリアし、新しいデータを書き込み
    worksheet.clear()
    worksheet.update("A1", data)

    print("スプレッドシート「選考締め切り情報」シート「外資就活」に CSV の内容が更新されました。")

if __name__ == "__main__":
    update_spreadsheet_from_csv()